# 1. 연구목표

- MNIST 손글씨 이미지 데이터를 사용
- 손글씨 이미지를 분류!!
- 알고리즘 목적
  - 딥러닝의 CNN을 이용하여서 손글씨를 예측하는 모델을 구축하겟다
  - 예측 모델나오고, 정확도를 평가하여 관찰
- 딥러닝 엔진중 텐서플로우를 이용하여 구현한다

# 2. 데이터 확보/수집

In [0]:
import tensorflow as tf
%tensorflow_version 1.x

In [3]:
mnist = tf.keras.datasets.mnist.load_data(path='mnist.npz')

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
type(mnist), len(mnist[0][0]), mnist[0][1], len(mnist[1][0]), mnist[1][1]
# 훈련용데이터 : mnist[0][0]
# 훈련용레이블 : mnist[0][1]
# 테스트용데이터 : mnist[1][0]
# 테스트용레이블 : mnist[1][1]
# 레이블 데이터는 백터화가 않되 있고, 분류형으로 0~9까지 배치되어 있다

(tuple,
 60000,
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 10000,
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [9]:
# 레이블이 이미 백터화 되어서 제공하는 tf의 데이터셋을 활용
# 단, 1.x에서만 사용가능
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [10]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [11]:
# 이미 데이터는 이미지 데이터는 28x28 픽셀로 처리되었고
# 레이블은 0~9를 0과 1로 구성하는 백터화 처리(이진 데이터)로 구성되었다
mnist.train.images.shape, mnist.train.labels.shape

((55000, 784), (55000, 10))

In [23]:
mnist.test.images.shape, mnist.test.labels.shape

((10000, 784), (10000, 10))

In [12]:
mnist.train.labels[:2]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [25]:
# 이미지 데이터는 float이다
mnist.train.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 3. 데이터 준비

In [17]:
# 백터화된 레이블을 보고 원래값 0~9를 찾아내는 방법 확인
# 힌트 => 배열 10칸중에서 1이 존재하는 인덱스값 => 실제 수치값
import numpy as np
np.where( mnist.train.labels[:1][0] )[0][0]

7

In [19]:
# 이미지 1개당 피쳐(feature)의 크기
pixels = mnist.train.images.shape[1]
pixels

784

In [20]:
# 레이블 1개당 피쳐(feature)의 크기
nums = mnist.train.labels.shape[1]
nums

10

In [22]:
# 이미지 가로크기 혹은 세로크기(정사각형이므로 동일)
pixel_wh_size = int(np.sqrt( pixels ))
pixel_wh_size

28

# 4. 데이터 분석(생략)

# 5. 모델 구축, 모델링

## 5-1. 데이터 플로우 그래프 구축
- 상수, 변수, 플레이스홀더, 연산등의 요소를 레이어를 구축하면서 정의(Define)
- Layer 구축 및 연결(원칙:앞단계의 출력이 뒷단계의 입력이다)
  - 입력층
  - 합성곱층1
  - 풀링층1
  - 합성곱층2
  - 풀링층2
  - 전결합층
  - 드롭아웃층
  - 출력층

## 5-2. 실행(학습, 예측, 평가)
- 데이터를 준비(batch사이즈,..)->학습->예측->평가(출력)

## 입력층

In [0]:
# x
# 손글씨 이미지 데이터가 들어오는 플레이스 홀더
# 외부에서 학습시 데이터가 주입된다
# 이미지 1개를 표현하는데 데이터가 784개의 feature가 필요
# 이런 이미지가 여러개 주입될수 있다 => n개 
# shape : ( n , 784 )
x = tf.placeholder( tf.float32, shape=( None, pixels ), name='x' )

## 합성곱층1

### W
- 가중치를 파라미터로 가지는 필터값 필요
- 행렬, 외부에서 주입(X), 내부적인 함수를 활용해서 생성한다 => 텐서의 타입중 Variable
- 2개의 합성곱층에서 W가 각각 필요하다 => 2군데 이상 코드에서 W가 필요하다 => W를 만드는 함수가 필요하다

In [0]:
# 가중치를 초기화 하고 W를 생성해서 리턴하는 함수를 구현
# shape : 가중치를 공용파라미터로 가지는 필터(=커널)의 shape
#         커널의 크기가 무조건 3x3은 아니다
# name  : 각 구성원들의 이름을 구분하기위해서 자동생성이 아닌 직접 부여
def makeWeightVariable( shape, name ):
  # 변수의 초기값 -> 절단평규분포를 통한 난수를 발생하는 함수를 활용(설정)
  # shape 이 크기에 맞춰서 배열을 만들고, 함수가 계산해서 난수를 배치
  # stddev : 표준편차
  # shape 크기대로 난수가 배치되었다(커널 크기만큼 행렬이 만들어지고, 난수가 설정되었다)
  W_initValue = tf.truncated_normal( shape, stddev=0.1 )
  # 변수 생성
  W = tf.Variable( W_initValue, name='W_' + name ) 
  return W

In [0]:
# 바이어스(편향) 초기화 함수
# 변수로 정의하고, 고정값(임계값으로 0.1)을 임시 부여
def makeBiasVariable( shape, name ):
  b_initValue = tf.constant( 0.1, shape=[shape] )
  b = tf.Variable( b_initValue, name='b_'+name )
  return b


In [43]:
# 합성곱 계층을 만드는 함수
def makeConv2d( x, W ):
  # 스트라이드는 고정해서 입력(설정)
  conv2d = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')  
  return conv2d
'''
tf.nn.conv2d(
    input : x => [batch(1회 훈련하고자 이미지의 총개수), 
                  in_height(세로크기), in_width(가로크기), 
                  in_channels(칼라에따라 다름, 1or 3) ]
    filters : 필터=커널, 필터의 공용파라미터:필터를 구성하는 구성원의 값 W
              [filter_height, filter_width : 필터의 세로, 가로 크기 내맘대로 정한다(상황고려)
              in_channels : 입력 채널수 -> 1
              out_channels: 출력 채널수 -> 본인이 설정(몇개의 이미지를 생성할것이지)]
    strides : 정수값, 정수값의 리스트로만 구성, length 1, 2 or 4
              [batch , w , h, depth ]
              통상적으로 batch, depth => 1을 사용
              w와 h는 일반적으로 같은값 부여
              [1,1,1,1] => 
    padding : 특성맵을 보정한다. 자르다보면 외곽선 부분에서 개수가 부족하게 나올수도 있다
              이를 통상 0으로 테두리를 치면서 보정, 
              SAME이란 값을 적용
    data_format='NHWC',
    dilations=None,
    name=None
)
'''

"\ntf.nn.conv2d(\n    input : x => [batch(1회 훈련하고자 이미지의 총개수), \n                  in_height(세로크기), in_width(가로크기), \n                  in_channels(칼라에따라 다름, 1or 3) ]\n    filters : 필터=커널, 필터의 공용파라미터:필터를 구성하는 구성원의 값 W\n              [filter_height, filter_width : 필터의 세로, 가로 크기 내맘대로 정한다(상황고려)\n              in_channels : 입력 채널수 -> 1\n              out_channels: 출력 채널수 -> 본인이 설정(몇개의 이미지를 생성할것이지)]\n    strides : 정수값, 정수값의 리스트로만 구성, length 1, 2 or 4\n              [batch , w , h, depth ]\n              통상적으로 batch, depth => 1을 사용\n              w와 h는 일반적으로 같은값 부여\n              [1,1,1,1] => \n    padding : 특성맵을 보정한다. 자르다보면 외곽선 부분에서 개수가 부족하게 나올수도 있다\n              이를 통상 0으로 테두리를 치면서 보정, \n              SAME이란 값을 적용\n    data_format='NHWC',\n    dilations=None,\n    name=None\n)\n"

In [0]:
# 합성곱층 1 생성 : 텐서보드상에 작업공간 잡을렬고
with tf.name_scope('conv1') as scope:
  # W
  # shape= [필터세로크기,필터가로크기,입력채널수,출력채널수]
  # 설정값 : 5, 5, 32 => 5x5 필터를 원본이미지에서 스트라이드해서 
  # 나온 피쳐맵이 총 32개가된다
  W_conv1 = makeWeightVariable( [ 5, 5, 1, 32 ], name='conv1' )
  
  # b
  # 가중치 필터의 shape의 마지막값과 편향의 shape값은 동일하다
  b_conv1 = makeBiasVariable( 32 , '' )

  # x => [ batch , h , w, channels]
  # x=[ None, 784 ] => [ batch , h , w, channels]
  # pixel_wh_size => 28(세로, 가로)
  x_image = tf.reshape( x, [ -1, pixel_wh_size, pixel_wh_size, 1] )

  # 컨볼류션 레이어 생성
  # 활성화 함수를 통과시켜서 레이어의 출력물(특성맵)을 비선형으로 보정처리한다
  # 이렇게 되면 특성맵이 활성화맵으로 되었다라고 표현할수 있다
  h_conv1 = tf.nn.relu( makeConv2d( x_image, W_conv1 ) + b_conv1 )
  # h_conv1은 합성곱층1의 출력물, 활성화맵, 다른 풀링층1의 입력데이터가 된다

In [45]:
h_conv1
# 합성곱층을 통과했을때 특성맵의 크기는 원본과 같을수도(SAME), 다를수도(VAILD) 있다
# 크기를 결정하는 요인으로 => stride, padding
# 합성곱의 핵심을 원본 이미지의 특성을 추출한 이미지가 출력채널수만큼 생성된다는 점
# padding에 의해서 채워질때, 원본가 동일한 사이즈로 가는가? => 크기가 동일하다
# padding을 SAME으로 하면 => (?, 28, 28, 32)
# 유효한 부분만 가는가? 크기가 줄어든다
# padding을 VALID로  하면 => (?, 24, 24, 32) 

<tf.Tensor 'conv1_4/Relu:0' shape=(?, 28, 28, 32) dtype=float32>

In [46]:
h_conv1.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)])

## 풀링층1

- 특성맵(or 활성화맵)의 특성을 강화
- 최대풀링, 평균풀링, 최소풀링등등 여러개 값중 하나를 취해서 행렬을 새로 구성(축소)
- 샘플링한다라고 표현
- 크기에 관여 => 얼마큼 이동시키면서 처리할것인가? => stride, 커널에는 파라미터는 없고(최대, 평균, 최소등 의미)
- 입력 : 이전 계층 합성곱층1의 결과물 => h_conv1

In [47]:
def makeMaxPooling( x ):
  return tf.nn.max_pool( x, ksize=[1 ,2 ,2 ,1 ],strides=[1,2,2,1], padding='SAME' )
'''
  tf.nn.max_pool(
    value:이전 계층의 출력값, [batch , h, w, channels ]
    ksize:최대값풀링을 수행하기 위한 슬라이딩 윈도우(= 커널, 필터)의 크기
          [ , h, w, ]
    strides: ksize의 이동 보폭, 몇칸씩 이동할 것인가
          [ batch , w , h, depth  ]
    padding: 보정, 크기가 않맞으면, SAME, ..
        
  )
'''

'\n  tf.nn.max_pool(\n    value:이전 계층의 출력값, [batch , h, w, channels ]\n    ksize:최대값풀링을 수행하기 위한 슬라이딩 윈도우(= 커널, 필터)의 크기\n          [ , h, w, ]\n    strides: ksize의 이동 보폭, 몇칸씩 이동할 것인가\n          [ batch , w , h, depth  ]\n    padding: 보정, 크기가 않맞으면, SAME, ..\n        \n  )\n'

In [0]:
# 풀링층 1 생성
with tf.name_scope('pool1') as scope:
  h_pool1 = makeMaxPooling( h_conv1 )

In [49]:
h_pool1

<tf.Tensor 'pool1/MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

## 합성곱층2

## 풀링층2

## 전결합층

## 드롭아웃층

## 출력층

# 6. 시스템 통합(생략)